## GLACIER AREA CALCULATOR ### 

This project will primarily use the USGS Python Package to identify, download and present Landsat imagery from 1985, 2005 and 2025 across several glacier mountain ranges across the world. 
The mountain ranges will include:
- Kenai Mountains (Alaska)
- Swiss Alps
- *Insert Greenland/Himalayas maybe?*
This script will download and present the images in False Colour Composite, then will generate a classification to identify the ice area on the images and this classification will be used in each image to calculate the change in glacier area across glaciated areas on a global scale.

##TO DO LIST FOR THE STUDY## 


----KENAI MOUNTAINS----
- Identify Landsat information on the USGS EE site and get the correct WRS path, etc.
- Download Landsat Images for Kenai Mountains first
- Clasification
- Area change calculation
- See if you can edit the image to show only the selected mountain range (Crop Stacked Landsat TIF)
- Maybe include some help() but ask Bob if it is something that is expected or required

----Swiss Alps-----
- Same as above
- Try to select area with Alestchgletscher

----GREENLAND/HIMALAYAS----
- Same as above


---AFTER LANDSAT DONE-----
- Generate a comparison Table/Template for all results acquired
- IF YOU HAVE TIME, maybe do a couple more glacier mountains
- Keep it that the script can be used to generate area as long as the correct info is input (ID, WRS, ETC..)
- Ask if an ArcGIS element is expected for the assignment
- Ask/Check how the how-to-guide is to be formatted (separate word doc in Git Repos/in the README/....
- Check how the final repos is to be submitted



In [1]:
import usgs

In [2]:
from usgs import api

In [6]:
# Open and read the Username and Api Key/Token for the USGS account
with open('C:/Users/couse/.usgs_user', 'r') as usertext:
    username = usertext.read()
with open('C:/Users/couse/.usgs_token', 'r') as tokentext:
    password = tokentext.read()
    
# Log in using the above information in the script attached below
login = api.login(username, password, save=True)

#The API Key (token) is extracted separately as it will be used in the script to search for and download Landsat imagery
api_key = login['data']

To identify and search for the required Landsat scenes in this study we are going to use the api.scene_search() function as this will be the primary method for getting the correct images before we download them. Below is some of the information we will need for the search

In [10]:
help(api.scene_search)

Help on function scene_search in module usgs.api:

scene_search(dataset, max_results=5000, metadata_type=None, start_date=None, end_date=None, ll=None, ur=None, lat=None, lng=None, distance=100, where=None, starting_number=1, sort_order='DESC', api_key=None)
    :param dataset:
        USGS dataset (e.g. EO1_HYP_PUB, LANDSAT_8)
    :param lat:
        Latitude
    :param lng:
        Longitude
    :param distance:
        Distance in meters used to for a radial search
    :param ll:
        Dictionary of longitude/latitude coordinates for the lower left corner
        of a bounding box search. e.g. { "longitude": 0.0, "latitude": 0.0 }
    :param ur:
        Dictionary of longitude/latitude coordinates for the upper right corner
        of a bounding box search. e.g. { "longitude": 0.0, "latitude": 0.0 }
    :param start_date:
        Start date for when a scene has been acquired
    :param end_date:
        End date for when a scene has been acquired
    :where:
        Dictionary rep

For the scene searches in this study we are going to need the following:
- Dataset ID (dataset)
- Latitude and Longitude (lat and lng)
- Earth Explorer allows us to create a bounding box to get available images that cover those areas (ll and ur - Lower left corner and Upper Right corner for the bounding box respectively)
- Start and End date (start_date and end_date)
- Max Results incase there are multiple available images (max_results)
- The extracted Api key (api_key)

Future adaptations could include filtering image selection based on cloud coverage to ensure that images used are cloud free.

We are going to use dataset_search to identify what Landsat datasets are available and importantly the exact 'dataset' which we will use to download and extract the specific scenes needed for this study

In [7]:
# Landsat imagery comes from the EarthExplorer dataset (node), we will do a search and print out all Landsat datasets in the Earth Explorer node
satellite_avail = api.dataset_search("landsat", "EE", api_key=api_key)
for dataset in satellite_avail['data']:
        print(dataset['datasetAlias'])

esat_etm_nopan
esat_etm_pan
ortho_mosaic_etm
geos_5_fp_it
geos_5_it
lima
landsat_mss_c2_l1
landsat_tm_c2_l1
landsat_tm_c2_l2
landsat_ba_tile_c2
landsat_dswe_tile_c2
landsat_fsca_tile_c2
landsat_fsca_tile_stat_c2
landsat_ard_tile_c2
landsat_etm_c2_l1
landsat_etm_c2_l2
landsat_ot_c2_l1
landsat_ot_c2_l2
landsat_band_files_c2_l1
landsat_band_files_c2_l2
lima_mosaic
merra_2_c2
ortho_mss_scene
mss_film
rbv_film
sys_etm
esat_tm
tm_film
ortho_mosaic
viirs_atmos


All available scenes are printed above. In this project we are going to use the 'landsat_mss_c2_l1' (1985), 'landsat_tm_c2_l1' (~2004) and the 'landsat_ot_c2_l1' (2024) datasets to acquire imagery of the selected glacier mountainscapes.

In [11]:
# Logs out the existing USGS account for the script
api.logout()

{'requestId': 1984598599,
 'version': 'stable',
 'sessionId': 315040452,
 'data': None,
 'errorCode': None,
 'errorMessage': None}